<a href="https://colab.research.google.com/github/ymuto0302/base2_2020/blob/master/opendata2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オープンデータの活用(2)
東京都，神奈川県，北海道，大阪府等，多くの自治体が新型コロナウイルス(COVID-19)に関するデータをオープンデータとして公開している。

### 各自治体の提供するデータの形式
CSV形式であるが，項目名や文字コード(encoding)は様々である。
例えば東京都は utf-8，北海道は Shift JIS を文字コードとして用いる。

### 北海道の提供するデータを見てみよう
北海道は「北海道オープンデータカタログサイト」
https://www.harp.lg.jp/opendata/dataset/
にてデータを提供している。

このサイトにおいて，新型コロナウイルス感染症に関するデータは
https://www.harp.lg.jp/opendata/dataset/1369.html
に置かれている。この中から「陽性患者属性」をまとめた "patients.csv" を用いる。


```
No,リリース日,曜日,居住地,年代,性別,属性,備考,補足,退院,周囲の状況,濃厚接触者の状況,age_group,sex_en
1,2020-01-28T00:00,,中国武漢市,40代,女性,−,−,,,なし,濃厚接触者を特定し、健康観察終了,40,woman
2,2020-02-14T00:00,,札幌市,50代,男性,自営業,−,,,札幌市公表中,札幌市公表中,50,man
3,2020-02-19T00:00,,札幌市,40代,男性,会社員,−,,,札幌市公表中,札幌市公表中,40,man
4,2020-02-19T00:00,,七飯町,60代,男性,町議会議員,−,,,No.12,濃厚接触者を特定し、健康観察終了,60,man
5,2020-02-19T00:00,,札幌市,40代,男性,会社員,−,,,札幌市公表中,札幌市公表中,40,man
6,2020-02-21T00:00,,中富良野町,10歳未満,男性,小学生,−,,,No.7,濃厚接触者を特定し、健康観察終了,10,man
```

In [14]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

# CSV ファイルからデータ読み込み
# (注意) 北海度の提供するデータは Shift JIS で書かれているが，Pythonで処理する場合，
# (encoding="shift_jis" でなく) encoding="cp932"で指定しないとうまくいかないようだ。
df = pd.read_csv("/content/drive/My Drive/BASE2/OpenData/hokkaido_covid19_patients.csv", encoding="cp932")

In [16]:
df

,No,リリース日,曜日,居住地,年代,性別,属性,備考,補足,退院,周囲の状況,濃厚接触者の状況,age_group,sex_en
0,1,2020-01-28T00:00,NaN,中国武漢市,40代,女性,－,－,NaN,NaN,なし,濃厚接触者を特定し、健康観察終了,40,woman
1,2,2020-02-14T00:00,NaN,札幌市,50代,男性,自営業,－,NaN,NaN,札幌市公表中,札幌市公表中,50,man
2,3,2020-02-19T00:00,NaN,札幌市,40代,男性,会社員,－,NaN,NaN,札幌市公表中,札幌市公表中,40,man
3,4,2020-02-19T00:00,NaN,七飯町,60代,男性,町議会議員,－,NaN,NaN,No.12,濃厚接触者を特定し、健康観察終了,60,man
4,5,2020-02-19T00:00,NaN,札幌市,40代,男性,会社員,－,NaN,NaN,札幌市公表中,札幌市公表中,40,man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,1113,2020-06-06T00:00,NaN,札幌市,非公表,非公表,無職,軽症 会話可,NaN,NaN,札幌市公表中,札幌市公表中,0,-
1113,1114,2020-06-06T00:00,NaN,札幌市,60代,女性,自営業,軽症 会話可,NaN,NaN,札幌市公表中,札幌市公表中,60,woman
1114,1115,2020-06-06T00:00,NaN,札幌市,60代,非公表,非公表,軽症 会話可,NaN,NaN,札幌市公表中,札幌市公表中,60,-
1115,1116,2020-06-06T00:00,NaN,札幌市,70代,男性,無職,軽症 会話可,NaN,NaN,札幌市公表中,札幌市公表中,70,man


In [0]:
# リリース日（＝公表日），居住地，年代，性別のみを取り出す
df = df.loc[:, ["リリース日", "居住地", "年代", "性別"]]

In [24]:
df

,リリース日,居住地,年代,性別
0,2020-01-28T00:00,中国武漢市,40代,女性
1,2020-02-14T00:00,札幌市,50代,男性
2,2020-02-19T00:00,札幌市,40代,男性
3,2020-02-19T00:00,七飯町,60代,男性
4,2020-02-19T00:00,札幌市,40代,男性
...,...,...,...,...
1112,2020-06-06T00:00,札幌市,非公表,非公表
1113,2020-06-06T00:00,札幌市,60代,女性
1114,2020-06-06T00:00,札幌市,60代,非公表
1115,2020-06-06T00:00,札幌市,70代,男性


In [0]:
# リリース日および年代でグループ化して，"患者数[6月1日, 40台]=3人" のような構造を作る
df["カウント"] = 1 # 後ほど可視化（グラフ化）する際に利用するため，個々の患者に 1 を割り振る
df_count = df.groupby(["リリース日", "年代"]).count().reset_index()

In [26]:
df_count

,リリース日,年代,居住地,性別,カウント
0,2020-01-28T00:00,40代,1,1,1
1,2020-02-14T00:00,50代,1,1,1
2,2020-02-19T00:00,40代,2,2,2
3,2020-02-19T00:00,60代,1,1,1
4,2020-02-21T00:00,10代,1,1,1
...,...,...,...,...,...
495,2020-06-06T00:00,60代,2,2,2
496,2020-06-06T00:00,70代,1,1,1
497,2020-06-06T00:00,80代,1,1,1
498,2020-06-06T00:00,90代,1,1,1


In [22]:
# 可視化の準備
# matplotlib の機能のみではコードが長くなるため，PlotlyExpress を利用する
# 次行の "!pip ・・・・" は PlotlyExpress をインストールする手順
!pip install plotly_express

In [29]:
# 可視化
import plotly_express as px
fig = px.bar(df_count, x="リリース日", y="カウント", color="年代",
             title="新型コロナウイルス感染症に関するデータ【北海道】：陽性患者属性")
fig.show()

以上のコードをまとめた結果を以下に示す。

ここではデータフレーム（変数 df や df_count）を表示していない。

In [30]:
'''
新型コロナウイルス感染症に関するデータ【北海道】の利用
'''

# CSV ファイルからデータ読み込み
# (注意) 北海度の提供するデータは Shift JIS で書かれているが，Pythonで処理する場合，
# (encoding="shift_jis" でなく) encoding="cp932"で指定しないとうまくいかないようだ。
df = pd.read_csv("/content/drive/My Drive/BASE2/OpenData/hokkaido_covid19_patients.csv", encoding="cp932")

# リリース日（＝公表日），居住地，年代，性別のみを取り出す
df = df.loc[:, ["リリース日", "居住地", "年代", "性別"]]

# リリース日および年代でグループ化して，"患者数[6月1日, 40台]=3人" のような構造を作る
df["カウント"] = 1 # 後ほど可視化（グラフ化）する際に利用するため，個々の患者に 1 を割り振る
df_count = df.groupby(["リリース日", "年代"]).count().reset_index()

# pandas，内部的な matplotlib ではうまくいかない
#df_count.plot.bar(x="公表_年月日", y="count", stacked=True, cmap='Reds')
#plt.show()

# 可視化
import plotly_express as px
fig = px.bar(df_count, x="リリース日", y="カウント", color="年代",
      title="新型コロナウイルス感染症に関するデータ【北海道】：陽性患者属性")
fig.show()